<a href="https://colab.research.google.com/github/Gleidistony/Trabalho-Ci-ncias-de-Dados/blob/main/quintaquestao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**5 QUESTÃO**

a) inserir(valor): Insere um valor na árvore B

In [ ]:
import time
from collections import deque

In [ ]:
class BTreeNode:
    def __init__(self, t, folha=False):
        self.t = t  # Grau mínimo
        self.chaves = []  # Lista de chaves
        self.filhos = []  # Lista de filhos
        self.folha = folha  # Booleano indicando se é folha

    def inserir_nao_cheio(self, valor):
        i = len(self.chaves) - 1

        if self.folha:
            self.chaves.append(None)
            while i >= 0 and valor < self.chaves[i]:
                self.chaves[i + 1] = self.chaves[i]
                i -= 1
            self.chaves[i + 1] = valor
        else:
            while i >= 0 and valor < self.chaves[i]:
                i -= 1
            i += 1

            if len(self.filhos[i].chaves) == 2 * self.t - 1:
                self.dividir_filho(i, self.filhos[i])
                if valor > self.chaves[i]:
                    i += 1
            self.filhos[i].inserir_nao_cheio(valor)

    def dividir_filho(self, i, y):
        t = self.t
        z = BTreeNode(t, y.folha)
        z.chaves = y.chaves[t:]
        y.chaves = y.chaves[:t - 1]

        if not y.folha:
            z.filhos = y.filhos[t:]
            y.filhos = y.filhos[:t]

        self.filhos.insert(i + 1, z)
        self.chaves.insert(i, y.chaves.pop())


class BTree:
    def __init__(self, t):
        self.raiz = BTreeNode(t, True)
        self.t = t

    def inserir(self, valor):
        raiz = self.raiz

        if len(raiz.chaves) == 2 * self.t - 1:
            s = BTreeNode(self.t, False)
            s.filhos.insert(0, raiz)
            s.dividir_filho(0, raiz)
            i = 0
            if s.chaves[0] < valor:
                i += 1
            s.filhos[i].inserir_nao_cheio(valor)
            self.raiz = s
        else:
            raiz.inserir_nao_cheio(valor)

# Medição do tempo de inicialização de deques
def medir_tempo_deque(tamanho):
    inicio = time.time()
    d = deque(range(tamanho))
    fim = time.time()
    print(f"Tamanho do deque: {tamanho}")
    print(f"Tempo de inicialização: {fim - inicio:.6f} segundos\n")

if __name__ == '__main__':
    tamanhos = [1000, 10000, 1000000]
    for tamanho in tamanhos:
        medir_tempo_deque(tamanho)

Tamanho do deque: 1000
Tempo de inicialização: 0.000053 segundos

Tamanho do deque: 10000
Tempo de inicialização: 0.000444 segundos

Tamanho do deque: 1000000
Tempo de inicialização: 0.080835 segundos



Função: O código mede o tempo de inicialização de um deque com diferentes tamanhos usando deque(range(tamanho)).

Complexidade:

Tempo: O(n) — A inicialização percorre e insere n elementos no deque.

Espaço: O(n) — O deque precisa armazenar todos os n elementos criados.

O código apenas avalia o custo de criação do deque sem realizar operações adicionais (como inserções ou remoções).

Demonstração da implementação de uma árvore B em Python, juntamente com a medição do tempo de inicialização de deques de diferentes tamanhos.

b) remover(valor): Remove um valor da árvore B.

In [ ]:
import time
from collections import deque

In [ ]:
import time
from collections import deque

class BTreeNode:
    def __init__(self, t, leaf=False):
        self.t = t  # Grau mínimo
        self.leaf = leaf
        self.keys = []
        self.children = []

    def find_key(self, k):
        idx = 0
        while idx < len(self.keys) and self.keys[idx] < k:
            idx += 1
        return idx

    def remove(self, k):
        idx = self.find_key(k)

        if idx < len(self.keys) and self.keys[idx] == k:
            if self.leaf:
                self.keys.pop(idx)
            else:
                self.remove_internal_node(k, idx)
        else:
            if self.leaf:
                return  # A chave não está na árvore
            flag = (idx == len(self.keys))
            if len(self.children[idx].keys) < self.t:
                self.fill(idx)
            if flag and idx > len(self.keys):
                self.children[idx - 1].remove(k)
            else:
                self.children[idx].remove(k)

    def remove_internal_node(self, k, idx):
        if len(self.children[idx].keys) >= self.t:
            pred = self.get_predecessor(idx)
            self.keys[idx] = pred
            self.children[idx].remove(pred)
        elif len(self.children[idx + 1].keys) >= self.t:
            succ = self.get_successor(idx)
            self.keys[idx] = succ
            self.children[idx + 1].remove(succ)
        else:
            self.merge(idx)
            self.children[idx].remove(k)

    def get_predecessor(self, idx):
        current = self.children[idx]
        while not current.leaf:
            current = current.children[-1]
        return current.keys[-1]

    def get_successor(self, idx):
        current = self.children[idx + 1]
        while not current.leaf:
            current = current.children[0]
        return current.keys[0]

    def fill(self, idx):
        if idx != 0 and len(self.children[idx - 1].keys) >= self.t:
            self.borrow_from_prev(idx)
        elif idx != len(self.keys) and len(self.children[idx + 1].keys) >= self.t:
            self.borrow_from_next(idx)
        else:
            if idx != len(self.keys):
                self.merge(idx)
            else:
                self.merge(idx - 1)

    def borrow_from_prev(self, idx):
        child = self.children[idx]
        sibling = self.children[idx - 1]

        child.keys.insert(0, self.keys[idx - 1])
        if not child.leaf:
            child.children.insert(0, sibling.children.pop())
        self.keys[idx - 1] = sibling.keys.pop()

    def borrow_from_next(self, idx):
        child = self.children[idx]
        sibling = self.children[idx + 1]

        child.keys.append(self.keys[idx])
        if not child.leaf:
            child.children.append(sibling.children.pop(0))
        self.keys[idx] = sibling.keys.pop(0)

    def merge(self, idx):
        child = self.children[idx]
        sibling = self.children[idx + 1]

        child.keys.append(self.keys.pop(idx))
        child.keys.extend(sibling.keys)

        if not child.leaf:
            child.children.extend(sibling.children)

        self.children.pop(idx + 1)

class BTree:
    def __init__(self, t):
        self.t = t
        self.root = BTreeNode(t, True)

    def remove(self, k):
        if not self.root:
            return

        start_time = time.time()
        self.root.remove(k)
        end_time = time.time()

        if len(self.root.keys) == 0:
            if self.root.leaf:
                self.root = None
            else:
                self.root = self.root.children[0]
        print(f"Tempo para remover {k}: {end_time - start_time:.6f} segundos")

# Testando inicialização de deque
sizes = [1000, 10000, 1000000]
for size in sizes:
    start_time = time.time()
    dq = deque(range(size))
    end_time = time.time()
    print(f"Tempo de inicialização: {end_time - start_time:.6f} segundos")
    print(f"Tamanho do deque: {size}\n")

Tempo de inicialização: 0.000027 segundos
Tamanho do deque: 1000

Tempo de inicialização: 0.000352 segundos
Tamanho do deque: 10000

Tempo de inicialização: 0.034439 segundos
Tamanho do deque: 1000000



• Busca/Remoção/Inserção: O(log n)

A altura da Árvore B é logarítmica devido ao balanceamento.

Operações de remoção podem envolver fusões e redistribuições, mas o custo médio permanece O(log n).

• Inicialização: O(n)

Criar um deque com n elementos é linear em relação ao tamanho.

Inserções e remoções nas extremidades do deque são O(1).

c) altura(): Retorna a altura da árvore B.


In [ ]:
import time
from collections import deque

In [ ]:
class BTreeNode:
    def __init__(self, t, leaf=False):
        self.t = t
        self.leaf = leaf
        self.keys = []
        self.children = []

class BTree:
    def __init__(self, t):
        self.t = t
        self.root = BTreeNode(t, True)

    def altura(self):
        start_time = time.time()
        height = 0
        current = self.root
        while current and not current.leaf:
            current = current.children[0]
            height += 1
        end_time = time.time()
        print(f"Altura da árvore: {height}")
        print(f"Tempo para calcular altura: {end_time - start_time:.6f} segundos")
        return height

# Testando inicialização de deque
sizes = [1000, 10000, 1000000]
for size in sizes:
    start_time = time.time()
    dq = deque(range(size))
    end_time = time.time()
    print(f"Tempo de inicialização: {end_time - start_time:.6f} segundos")
    print(f"Tamanho do deque: {size}\n")

Tempo de inicialização: 0.000026 segundos
Tamanho do deque: 1000

Tempo de inicialização: 0.000527 segundos
Tamanho do deque: 10000

Tempo de inicialização: 0.044112 segundos
Tamanho do deque: 1000000



• Árvore B - Altura:

- Complexidade: O(h) onde h é a altura da árvore.

- Tempo: Muito rápido em árvores balanceadas (O(log n) em árvores B típicas).

• Deque:

- Inicialização: O(n) para criar um deque com n elementos.

- Operações em Extremidades: Inserções e remoções são O(1).

d) buscar(valor): Verifica se um valor está na árvore B.


In [ ]:
import time
from collections import deque

In [ ]:
class BTreeNode:
    def __init__(self, t, leaf=False):
        self.t = t  # Grau mínimo
        self.leaf = leaf
        self.keys = []
        self.children = []

    def search(self, k):
        i = 0
        while i < len(self.keys) and k > self.keys[i]:
            i += 1

        if i < len(self.keys) and self.keys[i] == k:
            return self

        if self.leaf:
            return None

        return self.children[i].search(k)

class BTree:
    def __init__(self, t):
        self.root = BTreeNode(t, True)
        self.t = t

    def search(self, k):
        return self.root.search(k)

# Função para medir o tempo de inicialização do deque
def measure_deque_initialization(sizes):
    for size in sizes:
        start_time = time.time()
        dq = deque(range(size))
        end_time = time.time()
        print(f"Tempo de inicialização: {end_time - start_time:.6f} segundos")
        print(f"Tamanho do deque: {size}\n")

# Testando o deque
sizes = [1000, 10000, 1000000]
measure_deque_initialization(sizes)

# Exemplo de uso da árvore B
btree = BTree(t=3)  # Grau mínimo 3
# Aqui você pode implementar a inserção e testar a busca
# Exemplo: node = btree.search(10)  # Buscar pelo valor 10


Tempo de inicialização: 0.000050 segundos
Tamanho do deque: 1000

Tempo de inicialização: 0.000225 segundos
Tamanho do deque: 10000

Tempo de inicialização: 0.044464 segundos
Tamanho do deque: 1000000



A árvore B tem a função search(valor) para verificar se um valor está presente.

A função "measure_deque_initialization" mede o tempo necessário para inicializar o deque em diferentes tamanhos.



1 - Busca na Árvore B:

• Complexidade: O(log n)

A busca é feita percorrendo as chaves e descendo recursivamente nos filhos.
Em um nó com t chaves, a busca interna custa O(t), mas a profundidade é O(logₜ n).

2 - Inicialização do Deque:

• Complexidade: O(n)
Inicializar o deque com n elementos é linear, pois cada elemento é inserido uma vez.

e) sucessor(x): Sucessor de um nó x na árvore.

In [ ]:
import time
from collections import deque

In [ ]:
class BTreeNode:
    def __init__(self, t, leaf=False):
        self.t = t
        self.leaf = leaf
        self.keys = []
        self.children = []

class BTree:
    def __init__(self, t):
        self.root = BTreeNode(t, True)
        self.t = t

    def insert(self, k):
        root = self.root
        if len(root.keys) == 2 * self.t - 1:
            s = BTreeNode(self.t)
            s.children.insert(0, root)
            self._split_child(s, 0)
            self._insert_non_full(s, k)
            self.root = s
        else:
            self._insert_non_full(root, k)

    def _insert_non_full(self, node, k):
        i = len(node.keys) - 1
        if node.leaf:
            node.keys.append(None)
            while i >= 0 and k < node.keys[i]:
                node.keys[i + 1] = node.keys[i]
                i -= 1
            node.keys[i + 1] = k
        else:
            while i >= 0 and k < node.keys[i]:
                i -= 1
            i += 1
            if len(node.children[i].keys) == 2 * self.t - 1:
                self._split_child(node, i)
                if k > node.keys[i]:
                    i += 1
            self._insert_non_full(node.children[i], k)

    def _split_child(self, parent, i):
        t = self.t
        y = parent.children[i]
        z = BTreeNode(t, y.leaf)
        parent.children.insert(i + 1, z)
        parent.keys.insert(i, y.keys[t - 1])
        z.keys = y.keys[t:(2 * t - 1)]
        y.keys = y.keys[0:(t - 1)]
        if not y.leaf:
            z.children = y.children[t:(2 * t)]
            y.children = y.children[0:t]

    def successor(self, node, k):
        idx = 0
        while idx < len(node.keys) and node.keys[idx] < k:
            idx += 1

        if idx < len(node.keys) and node.keys[idx] == k:
            if not node.leaf:
                return self._min_value(node.children[idx + 1])
            else:
                if idx + 1 < len(node.keys):
                    return node.keys[idx + 1]
                else:
                    return None
        elif not node.leaf:
            return self.successor(node.children[idx], k)
        else:
            return None

    def _min_value(self, node):
        current = node
        while not current.leaf:
            current = current.children[0]
        return current.keys[0]

def deque_initialization_test(sizes):
    for size in sizes:
        start_time = time.time()
        dq = deque(range(size))
        end_time = time.time()
        print(f"Tamanho do deque: {size}")
        print(f"Tempo de inicialização: {end_time - start_time:.6f} segundos\n")

if __name__ == "__main__":
    btree = BTree(t=3)
    elements = [10, 20, 5, 6, 12, 30, 7, 17]
    for elem in elements:
        btree.insert(elem)

    key = 6
    succ = btree.successor(btree.root, key)
    print(f"Sucessor de {key}: {succ}\n")

    sizes = [1000, 10000, 1000000]
    deque_initialization_test(sizes)

Sucessor de 6: 7

Tamanho do deque: 1000
Tempo de inicialização: 0.000038 segundos

Tamanho do deque: 10000
Tempo de inicialização: 0.000197 segundos

Tamanho do deque: 1000000
Tempo de inicialização: 0.033123 segundos



1 - Árvore B:

• Inserção: O(log n) — devido ao balanceamento da árvore.

• Busca do sucessor: O(log n) — navegação pela árvore para encontrar o sucessor.

2 - Deque Initialization:

• Inicialização: O(n) — onde n é o número de elementos, pois todos os elementos precisam ser inseridos no deque.

• Tempo de execução: Cresce linearmente com o aumento do tamanho do deque.

f) predecessor(x): Predecessor de um nó x na árvore.

In [ ]:
import time
from collections import deque

In [ ]:
class BTreeNode:
    def __init__(self, t, leaf=False):
        self.t = t  # Grau mínimo
        self.leaf = leaf
        self.keys = []
        self.children = []

class BTree:
    def __init__(self, t):
        self.root = BTreeNode(t, True)
        self.t = t

    def search(self, k, node=None):
        if node is None:
            node = self.root

        i = 0
        while i < len(node.keys) and k > node.keys[i]:
            i += 1

        if i < len(node.keys) and node.keys[i] == k:
            return node, i

        if node.leaf:
            return None

        return self.search(k, node.children[i])

    def insert(self, k):
        root = self.root
        if len(root.keys) == 2 * self.t - 1:
            new_root = BTreeNode(self.t, False)
            new_root.children.insert(0, root)
            self.split_child(new_root, 0)
            self.insert_non_full(new_root, k)
            self.root = new_root
        else:
            self.insert_non_full(root, k)

    def insert_non_full(self, node, k):
        i = len(node.keys) - 1

        if node.leaf:
            node.keys.append(0)
            while i >= 0 and k < node.keys[i]:
                node.keys[i + 1] = node.keys[i]
                i -= 1
            node.keys[i + 1] = k
        else:
            while i >= 0 and k < node.keys[i]:
                i -= 1
            i += 1

            if len(node.children[i].keys) == 2 * self.t - 1:
                self.split_child(node, i)
                if k > node.keys[i]:
                    i += 1
            self.insert_non_full(node.children[i], k)

    def split_child(self, parent, i):
        t = self.t
        y = parent.children[i]
        z = BTreeNode(t, y.leaf)

        parent.children.insert(i + 1, z)
        parent.keys.insert(i, y.keys[t - 1])

        z.keys = y.keys[t:(2 * t - 1)]
        y.keys = y.keys[0:(t - 1)]

        if not y.leaf:
            z.children = y.children[t:(2 * t)]
            y.children = y.children[0:t]

    def predecessor(self, k):
        node_info = self.search(k)
        if not node_info:
            return None
        node, idx = node_info

        if not node.leaf:
            cur = node.children[idx]
            while not cur.leaf:
                cur = cur.children[-1]
            return cur.keys[-1]
        elif idx > 0:
            return node.keys[idx - 1]
        else:
            return None

    def successor(self, k):
        node_info = self.search(k)
        if not node_info:
            return None
        node, idx = node_info

        if not node.leaf:
            cur = node.children[idx + 1]
            while not cur.leaf:
                cur = cur.children[0]
            return cur.keys[0]
        elif idx < len(node.keys) - 1:
            return node.keys[idx + 1]
        else:
            return None

# Função para medir tempo de inicialização do deque
def measure_deque_initialization(sizes):
    for size in sizes:
        start_time = time.time()
        d = deque(range(size))
        end_time = time.time()
        print(f"Tempo de inicialização: {end_time - start_time:.6f}s | Tamanho do deque: {size}")

if __name__ == '__main__':
    # Testando B-Tree
    btree = BTree(3)
    for num in [10, 20, 5, 6, 12, 30, 7, 17]:
        btree.insert(num)

    print("Sucessor de 6:", btree.successor(6))
    print("Predecessor de 6:", btree.predecessor(6))

    # Testando deque
    sizes = [1000, 10000, 1000000]
    measure_deque_initialization(sizes)


Sucessor de 6: 7
Predecessor de 6: 5
Tempo de inicialização: 0.000098s | Tamanho do deque: 1000
Tempo de inicialização: 0.000445s | Tamanho do deque: 10000
Tempo de inicialização: 0.081910s | Tamanho do deque: 1000000


1 - Árvore B:

• Inserção: O(log n) – devido à altura balanceada da árvore.

• Busca: O(log n) – percorre níveis até encontrar o elemento.

• Sucessor/Predecessor: O(log n) – baseado na busca.

• Espaço: O(n) – cada elemento é armazenado uma única vez.

2 - Deque:

• Inicialização: O(n) – insere n elementos linearmente.

• Espaço: O(n) – armazena n elementos.

g) imprimirEmOrdem(): Imprime todos os valores da árvore B em ordem crescente

In [1]:
import time
import collections

In [2]:
class BTreeNode:
    def __init__(self, leaf=False):
        self.leaf = leaf
        self.keys = []
        self.children = []

class BTree:
    def __init__(self, t):
        self.root = BTreeNode(leaf=True)
        self.t = t  # Grau mínimo da árvore B

    def imprimirEmOrdem(self, node=None):
        if node is None:
            node = self.root

        for i in range(len(node.keys)):
            if not node.leaf:
                self.imprimirEmOrdem(node.children[i])
            print(node.keys[i], end=' ')

        if not node.leaf:
            self.imprimirEmOrdem(node.children[-1])

# Teste de inicialização do deque
def medir_tempo_deque(tamanho):
    inicio = time.time()
    d = collections.deque(maxlen=tamanho)
    fim = time.time()
    print(f"Tempo de inicialização: {fim - inicio:.6f} segundos | Tamanho do deque: {tamanho}")

# Executando os testes com deque
for tamanho in [1000, 10000, 1000000]:
    medir_tempo_deque(tamanho)

# Exemplo de uso da Árvore B
b_tree = BTree(t=3)
b_tree.root.keys = [10, 20, 30]
b_tree.root.children = [BTreeNode(leaf=True), BTreeNode(leaf=True), BTreeNode(leaf=True), BTreeNode(leaf=True)]
b_tree.root.children[0].keys = [5]
b_tree.root.children[1].keys = [15]
b_tree.root.children[2].keys = [25]
b_tree.root.children[3].keys = [35]

print("Elementos da Árvore B em ordem:")
b_tree.imprimirEmOrdem()


Tempo de inicialização: 0.000005 segundos | Tamanho do deque: 1000
Tempo de inicialização: 0.000002 segundos | Tamanho do deque: 10000
Tempo de inicialização: 0.000001 segundos | Tamanho do deque: 1000000
Elementos da Árvore B em ordem:
10 20 30 

1 - Impressão em ordem (imprimirEmOrdem())

• Complexidade:
𝑂(𝑛)
O(n), onde 𝑛
n é o número de chaves na árvore. Isso ocorre porque percorremos todos os nós e imprimimos as chaves em ordem crescente.

• Tempo de execução: Depende do número total de chaves na árvore. No pior caso, é linear em relação ao tamanho da árvore.

2 - Inicialização do deque (medir_tempo_deque())
Complexidade:
𝑂
(
1
)
O(1), pois a criação de um deque com um tamanho fixo é uma operação de tempo constante.

• Tempo de execução: Medido experimentalmente no código, mas geralmente é muito pequeno (da ordem de microssegundos a milissegundos, dependendo do tamanho).

h) dividirNo(no): Divide um nó da árvore B quando ele excede o número máximo de
chaves. O nó é dividido em dois, e a chave do meio é promovida ao nó pai.


In [1]:
import time
from collections import deque

In [2]:
class NoB:
    def __init__(self, t, folha=False):
        self.t = t  # Grau mínimo
        self.folha = folha
        self.chaves = []  # Lista de chaves
        self.filhos = []  # Lista de filhos

class ArvoreB:
    def __init__(self, t):
        self.raiz = NoB(t, True)
        self.t = t

    def dividirNo(self, pai, indice, no):
        t = self.t
        novo_no = NoB(t, no.folha)
        meio = t - 1

        novo_no.chaves = no.chaves[meio + 1:]
        no.chaves = no.chaves[:meio]

        if not no.folha:
            novo_no.filhos = no.filhos[meio + 1:]
            no.filhos = no.filhos[:meio + 1]

        pai.filhos.insert(indice + 1, novo_no)
        pai.chaves.insert(indice, no.chaves.pop(-1))

# Função para medir o tempo de inicialização de um deque
def medir_tempo_deque(tamanho):
    inicio = time.time()
    d = deque(maxlen=tamanho)
    fim = time.time()
    print(f"Tempo de inicialização: {fim - inicio:.6f} segundos  Tamanho do deque: {tamanho}")

# Teste dos tempos de inicialização do deque
for tamanho in [1000, 10000, 1000000]:
    medir_tempo_deque(tamanho)


Tempo de inicialização: 0.000003 segundos  Tamanho do deque: 1000
Tempo de inicialização: 0.000002 segundos  Tamanho do deque: 10000
Tempo de inicialização: 0.000001 segundos  Tamanho do deque: 1000000


Divisão de um nó na Árvore B (dividirNo)

1 - Complexidade:
𝑂
(
𝑡
)
O(t), onde
𝑡
t é o grau mínimo da árvore.

• Motivo: A operação envolve copiar metade das chaves e filhos para um novo nó, o que leva tempo proporcional ao número de elementos.
Inicialização do deque (medir_tempo_deque)

2 - Complexidade:
𝑂
(
1
)
O(1) para criar um deque com um tamanho máximo especificado.

• Tempo de execução: O tempo de inicialização é quase constante, independentemente do tamanho do deque, pois a memória só é alocada quando elementos são adicionados.

i)Calcular a ordem da árvore (número máximo de filhos por nó).

In [4]:
import time
import collections

In [6]:
class BTreeNode:
    def __init__(self, leaf=False):
        self.leaf = leaf
        self.keys = []
        self.children = []

class BTree:
    def __init__(self, t):
        self.root = BTreeNode(True)
        self.t = t  # Grau mínimo da árvore B

    def get_order(self):
        return 2 * self.t  # Ordem da árvore B (número máximo de filhos por nó)

# Função para medir o tempo de inicialização do deque
def measure_deque_initialization(size):
    start_time = time.time()
    dq = deque(range(size))
    end_time = time.time()
    print(f"Tempo de inicialização: {end_time - start_time:.6f} segundos | Tamanho do deque: {size}")

if __name__ == "__main__":
    # Criando uma árvore B com grau mínimo t = 3
    b_tree = BTree(3)
    print(f"Ordem da árvore B: {b_tree.get_order()}")

    # Medindo tempo de inicialização para diferentes tamanhos de deque
    for size in [1000, 10000, 1000000]:
        measure_deque_initialization(size)

Ordem da árvore B: 6
Tempo de inicialização: 0.000032 segundos | Tamanho do deque: 1000
Tempo de inicialização: 0.000193 segundos | Tamanho do deque: 10000
Tempo de inicialização: 0.037040 segundos | Tamanho do deque: 1000000


Cálculo da ordem da árvore B (get_order)

1 - Complexidade:
𝑂
(
1
)
O(1) (Tempo constante, pois é apenas uma multiplicação).

• Tempo de execução: Muito rápido, praticamente instantâneo.
Inicialização do deque (measure_deque_initialization)

1 - Complexidade:
𝑂
(
𝑛
)
O(n) (Onde
𝑛
n é o tamanho do deque, pois ele percorre e adiciona elementos).

• Tempo de execução: Cresce linearmente com o tamanho do deque, sendo rápido para tamanhos pequenos e mais lento para grandes.

j) Implemente uma função que retorne o número total de chaves armazenadas na árvore
B.

In [7]:
import time
from collections import deque

In [8]:
class BTreeNode:
    def __init__(self, leaf=False):
        self.leaf = leaf
        self.keys = []
        self.children = []

class BTree:
    def __init__(self, t):
        self.root = BTreeNode(True)
        self.t = t

    def count_keys(self, node=None):
        if node is None:
            node = self.root
        count = len(node.keys)
        if not node.leaf:
            for child in node.children:
                count += self.count_keys(child)
        return count

def measure_deque_initialization(sizes):
    for size in sizes:
        start_time = time.time()
        d = deque(range(size))
        end_time = time.time()
        print(f"Tempo de inicialização: {end_time - start_time:.6f} segundos | Tamanho do deque: {size}")

# Criando e populando uma árvore B com ordem 3
b_tree = BTree(3)
print(f"Número total de chaves na árvore B: {b_tree.count_keys()}")

# Medindo tempo de inicialização do deque
measure_deque_initialization([1000, 10000, 1000000])


Número total de chaves na árvore B: 0
Tempo de inicialização: 0.000026 segundos | Tamanho do deque: 1000
Tempo de inicialização: 0.000210 segundos | Tamanho do deque: 10000
Tempo de inicialização: 0.042258 segundos | Tamanho do deque: 1000000


Contagem de chaves na Árvore B (count_keys)

1 - Complexidade:
𝑂
(
𝑛
)
O(n), onde
𝑛
n é o número total de chaves na árvore.

• Explicação: O algoritmo percorre todas as chaves armazenadas na árvore B, incluindo as dos nós filhos.

• Tempo de execução: Depende do tamanho da árvore, mas em um caso balanceado, o tempo de execução cresce linearmente com o número de chaves.

• Inicialização do deque (measure_deque_initialization)

2 - Complexidade:
𝑂
(
𝑛
)
O(n), onde
𝑛
n é o tamanho do deque.

• Explicação: Criar um deque(range(n)) exige a adição de
𝑛
n elementos, o que leva tempo proporcional a
𝑛
n.

• Tempo de execução: Aproximadamente linear em relação ao tamanho do deque, ou seja, dobra quando o tamanho dobra.